In [1]:
# import used libraries
import pandas as pd                        # pandas for data analysis
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt            # matplotlib for data visualisation
import json
import zstandard as zstd

In [2]:
### PATHS ###

DIR = "../data/"

#read
TIMESERIES_PATH = DIR + "df_timeseries_en.tsv.gz"
CHANNELS_PATH   = DIR + "df_channels_en.tsv.gz"

#write
ENT_TIMESERIES_PATH = DIR + "ent_timeseries_en.tsv.zip"
ENT_CHANNELS_PATH   = DIR + "ent_channels_en.tsv.zip"

In [3]:
### IMPORTS ###

timeseries = pd.read_csv(TIMESERIES_PATH, sep='\t')
channels   = pd.read_csv(CHANNELS_PATH, sep='\t')

timeseries['datetime'] = pd.to_datetime(timeseries['datetime'])
channels['join_date'] = pd.to_datetime(channels['join_date'])

In [4]:
### TREATMENT ###

#compute the number of subs at the start of the period for each youtuber 
channels = pd.merge(channels,
                    timeseries.sort_values(by='datetime').drop_duplicates('channel')[['channel','subs']].rename(columns={"subs":"initial_subs"}),
                    on = 'channel')

#keep only channels that started between 5k and 10k from the entertaining category
ent_channels = channels[(channels['initial_subs'] > 5e3) &
                        (channels['initial_subs'] < 10e3) &
                        (channels['category_cc'] == 'Entertainment')]

ent_timeseries = pd.merge(timeseries, ent_channels['channel'], on='channel').drop(['category'], axis=1)

#compute the weekly evolution for each channel in timeseries
ent_timeseries['evolution'] = ent_timeseries['delta_subs']/ent_timeseries['subs']

#compute the evolution score by taking the mean weekly evolution
evo_score = ent_timeseries.groupby('channel').mean()['evolution'].rename('evo_score')
ent_channels = pd.merge(ent_channels, evo_score, on='channel')

# #keeps only 25% top and 25% bottom channels
top_channels = ent_channels.nlargest(int(len(ent_channels)*0.25), 'evo_score')[['channel','evo_score']]
bottom_channels = ent_channels.nsmallest(int(len(ent_channels)*0.25), 'evo_score')[['channel','evo_score']]
evo_channels = pd.concat([top_channels, bottom_channels]).sort_values('evo_score', ascending=False)
evo_channels['has_buzzed'] = 0 + 1 * (evo_channels['evo_score'] > bottom_channels['evo_score'].max())

ent_channels = pd.merge(ent_channels, evo_channels.drop(['evo_score'], axis=1), on='channel')
ent_timeseries = pd.merge(ent_timeseries, ent_channels[['channel', 'has_buzzed']], on='channel')


In [5]:
### EXPORTS ###

ent_channels.to_csv(ENT_CHANNELS_PATH, index=False, compression={'method':'zip'})
ent_timeseries.to_csv(ENT_TIMESERIES_PATH, index=False, compression={'method':'zip'})

In [6]:
##################################################################################################
############################################ METADATA ############################################
##################################################################################################

In [7]:

############################################## PATH ##############################################

METADATA_PATH   = DIR + "_raw_yt_metadata.jsonl.zst"

In [8]:

############################## READ AND SPLIT INTO SMALLER CSV FILES #############################

class zreader:

    def __init__(self, file, chunk_size=16384):
        self.fh = open(file, 'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''

    def readlines(self):
        while True:
            chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]

reader = zreader(METADATA_PATH)
metadata = []
df_metadata = pd.DataFrame([])

idx = 0
store_idx = 0
save_idx = 0

min_upload_date, max_upload_date = pd.to_datetime("01-01-2015"), pd.to_datetime("09-30-2019")
for line in reader.readlines():
    line_dict = json.loads(line)
    
    if (pd.to_datetime(line_dict["upload_date"]) < max_upload_date) & \
       (pd.to_datetime(line_dict["upload_date"]) > min_upload_date) & \
       (line_dict['channel_id'] in ent_channels['channel'].values):
        
        del line_dict['crawl_date']
        del line_dict['categories']
        
        metadata.append(line_dict)
    idx += 1
    if idx%100000 == 0:
        print(idx)
    
    #store in a dataframe every 1 million
    if len(metadata) >= 1000000:
        if store_idx < 9 : print(" - STORE", store_idx)
        df_metadata = pd.concat([df_metadata, pd.DataFrame(metadata)])
        metadata = []
        store_idx += 1
        
        
    #save dataframe every 10 million
    if len(df_metadata) >= 10000000:
        print(" - SAVE ", save_idx)
        
        S_METADATA_PATH = DIR + "metadata/_raw_yt_metadata" + str(save_idx) + ".tsv.zip"
    
        df_metadata.to_csv(S_METADATA_PATH, index=False, compression={'method':'zip'})
        df_metadata = pd.DataFrame([])
        store_idx = 0
        save_idx += 1
        
if store_idx < 9 : print(" - STORE", store_idx)
df_metadata = pd.concat([df_metadata, pd.DataFrame(metadata)])
metadata = []
store_idx += 1

print(" - SAVE ", save_idx)
        
S_METADATA_PATH = DIR + "metadata/_raw_yt_metadata" + str(save_idx) + ".tsv.zip"

df_metadata.to_csv(S_METADATA_PATH, index=False, compression={'method':'zip'})
df_metadata = pd.DataFrame([])
store_idx = 0
save_idx += 1

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000


In [31]:
# TREAT THE 6 METADATA
for i in range(7):
    print("Start metadata ", i)
    
    # PATH
    METADATA_PATH = DIR + "metadata/_raw_yt_metadata" + str(i) + ".tsv.zip"
    ENT_METADATA_PATH = DIR + "metadata/ent_metadata" + str(i) + ".tsv.zip"
    print("Path done - ")
    
    # READ
    metadata = pd.read_csv(METADATA_PATH)
    print("Read done - ")
    
    # TREATMENT
    metadata = metadata.rename(columns={'channel_id':'channel'})
    ent_metadata = pd.merge(metadata, ent_channels['channel'])
    print("Treatment done - ")
    
    # WRITE
    ent_metadata.to_csv(ENT_METADATA_PATH, index=False, compression={'method':'zip'})
    print("Write done -")
    
    print("Done metadata ", i)

Start metadata  6
Path done - 
Read done - 
Treatment done - 
Write done -
Done metadata  6


In [32]:
# STORE ALL DATAFRAMES IN ONLY ONE
ent_metadata = pd.DataFrame()
for i in range(7):
    print("Start metadata ", i)
    
    # PATH
    METADATA_PATH = DIR + "metadata/ent_metadata" + str(i) + ".tsv.zip"
    print("Path done - ")
    
    # READ
    metadata = pd.read_csv(METADATA_PATH)
    print("Read done - ")
    
    # CONCAT
    ent_metadata = pd.concat([ent_metadata, metadata], ignore_index=True)
    print("Concat done - ")


# WRITE
ENT_METADATA_PATH = DIR + "ent_metadata_en.tsv.zip"
ent_metadata.to_csv(ENT_METADATA_PATH, index=False, compression={'method':'zip'})
print("Write done -")

Start metadata  0
Path done - 
Read done - 
Concat done - 
Start metadata  1
Path done - 
Read done - 
Concat done - 
Start metadata  2
Path done - 
Read done - 
Concat done - 
Start metadata  3
Path done - 
Read done - 
Concat done - 
Start metadata  4
Path done - 
Read done - 
Concat done - 
Start metadata  5
Path done - 
Read done - 
Concat done - 
Start metadata  6
Path done - 
Read done - 
Concat done - 
Write done -


In [33]:
# KEEP COMMON CHANNELS BETWEEN df_channels, df_metadata AND df_timeseries
# df_metadata HAS LESS CHANNELS THAN df_channels AND df_timeseries (SOME CHANNELS WITHOUT ANY VIDEO ?)

#dir
DIR = "../data/"

#read path
TIMESERIES_PATH = DIR + "ent_timeseries_en.tsv.zip"
CHANNELS_PATH   = DIR + "ent_channels_en.tsv.zip"
METADATA_PATH   = DIR + "ent_metadata_en.tsv.zip"

#imports
timeseries = pd.read_csv(TIMESERIES_PATH)
channels   = pd.read_csv(CHANNELS_PATH)
metadata   = pd.read_csv(METADATA_PATH)

#treatment
channel_ids = metadata[['channel']].drop_duplicates()
ent_channels = pd.merge(channels, channel_ids)
ent_timeseries = pd.merge(timeseries, channel_ids)

#write path
ENT_TIMESERIES_PATH = DIR + "ent_timeseries_en.tsv.zip"
ENT_CHANNELS_PATH   = DIR + "ent_channels_en.tsv.zip"

#exports
ent_channels.to_csv(ENT_CHANNELS_PATH, index=False, compression={'method':'zip'})
ent_timeseries.to_csv(ENT_TIMESERIES_PATH, index=False, compression={'method':'zip'})

In [34]:
ent_channels

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,initial_subs,evo_score,has_buzzed
0,Entertainment,2013-05-03,UCMxtWuqP6Ktk1tsSZSstaug,Haryanvi Maina,8830000,1619,666.0,2.0870,9043.500000,0.043476,1
1,Entertainment,2006-02-27,UCDfk8ogO6QGeJAYCY0QDzKw,Stephen Sharer,7230000,549,867.0,2.0870,8843.000000,0.039752,1
2,Entertainment,2015-11-24,UCAgx4HcQIYn9lM0rhtIuH9w,HZHtube Kids Fun,6570000,358,914.0,2.0870,6100.000000,0.040441,1
3,Entertainment,2015-06-13,UCQcf-d098cNGGl3bPGGjHBg,OLIE THE CUB,6820000,778,941.0,2.0870,5181.000000,0.042452,1
4,Entertainment,2017-04-26,UC977n5nXRn8MDuGQjwxkOLw,PC Bob and Mr Bean: ...,6730000,3377,943.0,2.0870,6468.000000,0.051014,1
...,...,...,...,...,...,...,...,...,...,...,...
1253,Entertainment,2010-06-04,UCswGzcZ3DcBXN5Pc3J7d9Mg,BlaakowDancer,10200,65,977320.0,53.1435,7284.625000,0.001790,0
1254,Entertainment,2011-03-01,UChN6SlSBv21jbp6n1XRSbJA,thesongadayproject,10100,2624,977759.0,53.1435,9963.000000,0.000297,0
1255,Entertainment,2012-12-17,UCOioqCYGI_pOUWGGD1cMPCQ,Taylor Swift Brasil,10100,257,978347.0,53.1435,5840.744186,0.002987,0
1256,Entertainment,2013-12-27,UCQV1dyQqE8dtgWU6U_Rg0WQ,CrazySuperGamer,10300,38,978503.0,53.1435,9130.250000,0.000869,0
